In [ ]:
import pandas as pd
from tqdm.notebook import tqdm_notebook
from py2neo import Graph, Node, Relationship, Graph, Subgraph

In [ ]:
graph = Graph("bolt://localhost:27687", auth=("neo4j", "12345678"))
# 删除全部节点，慎重
# graph.delete_all()

### 保存实体节点到Neo4J

In [ ]:
df_entity = pd.read_csv("./entity_v1.csv")
df_entity.columns = ["name", "label", "description"]
df_entity = df_entity.drop_duplicates(subset=["name"])

In [ ]:
for item in tqdm_notebook(df_entity.to_dict("records")):
    label = item.pop("label")
    node = Node(label, **item)
    graph.create(node)

### 保存关系到Neo4J

In [ ]:
df_relation = pd.read_csv("./relation_v1.csv")
df_relation = df_relation.drop_duplicates(subset=["source", "target"])
# df_entity.columns = ["name", "label", "description"]

relationships = []
for item in df_relation.to_dict("records"):
    source = graph.nodes.match(name=item.get("source")).first()
    target = graph.nodes.match(name=item.get("target")).first()
    description = item.get("description")
    relationship = item.get("relationship")
    if source and target:
        relationships.append(
            Relationship(source, relationship, target, description=description)
        )
Relations = Subgraph(relationships=relationships)   
graph.create(Relations)

------